In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Integrating kaggle with google collab.

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv /content/drive/MyDrive/Notebooks/kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
asinow/car-price-dataset                                Car Price Dataset                                 135KB  2025-01-26 19:53:28          16645        264  1.0              
mahmoudelhemaly/students-grading-dataset                Student Performance & Behavior Dataset            508KB  2025-02-17 17:38:46           1521         34  1.0              
vinothkannaece/sales-dataset                            sales dataset                                      27KB  2025-02-18 05:13:42           2395         41  1.0              
abdulmalik1518/mobiles-dataset-2025                     Mobiles Dataset (2025)                             20K

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download geolek/grayscale-face-images

Dataset URL: https://www.kaggle.com/datasets/geolek/grayscale-face-images
License(s): MIT
100% 4.65G/4.65G [00:46<00:00, 171MB/s]
100% 4.65G/4.65G [00:46<00:00, 106MB/s]


In [ ]:
!unzip grayscale-face-images.zip


Archive:  grayscale-face-images.zip
replace Grayscale Face images/test/anger/anger001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Training the model

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [ ]:
# Path to the dataset folder
base_dir = "/content/Grayscale Face images/train"

In [ ]:
# Emotion label mapping
emotion_map = {
    0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy',
    4: 'Sad', 5: 'Surprise', 6: 'Neutral'
}

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # Split into training and validation
)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = Sequential([
    # First convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    # Second convolutional layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    # Third convolutional layer
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    # Flattening and dense layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



Found 72038 images belonging to 8 classes.
Found 18005 images belonging to 8 classes.


In [25]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    verbose=1
)

Epoch 1/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 227s 190ms/step - accuracy: 0.1686 - loss: 2.0484 - val_accuracy: 0.2688 - val_loss: 1.9125
Epoch 2/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 185s 164ms/step - accuracy: 0.2816 - loss: 1.8835 - val_accuracy: 0.3156 - val_loss: 1.7829
Epoch 3/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 179s 159ms/step - accuracy: 0.3306 - loss: 1.7854 - val_accuracy: 0.3493 - val_loss: 1.7144
Epoch 4/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 179s 159ms/step - accuracy: 0.3516 - loss: 1.7296 - val_accuracy: 0.3403 - val_loss: 1.7143
Epoch 5/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 177s 157ms/step - accuracy: 0.3680 - loss: 1.7054 - val_accuracy: 0.3725 - val_loss: 1.6681
Epoch 6/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 174s 155ms/step - accuracy: 0.3761 - loss: 1.6802 - val_accuracy: 0.3733 - val_loss: 1.6453
Epoch 7/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 174s 154ms/step - accuracy: 0.3848 - loss: 1.6574 - val_accuracy: 0.3808 - val_loss: 1.6299
Epoch 8/30
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 174s 154ms/step - ac

In [26]:
# Evaluate the model
eval_result = model.evaluate(validation_generator, verbose=0)
print(f"Validation Accuracy: {eval_result[1]:.2f}")

# Save the trained model
model.save("face_model.h5")
print("Model saved as 'face_emotion_detection_model.h5'")


Validation Accuracy: 0.45
Model saved as 'face_emotion_detection_model.h5'


Accessing Cam and Using the trained model

In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# Load the pre-trained model
model = load_model(r"D:\Mental Health\models\face_model.h5")

# Define the emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load the face detection model
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract the face region
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # Make predictions on the ROI using the model
            preds = model.predict(roi)[0]

            # Lookup the class
            label = emotion_labels[preds.argmax()]
            label_position = (x, y - 10)
            accuracy = np.max(preds) * 100

            # Display the label and accuracy on the frame
            cv2.putText(frame, f'{label} ({accuracy:.2f}%)', label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('Emotion Detector', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


: 

Text Summerization and Sentiment

In [1]:
from transformers import pipeline

def create_summarizer():
    """
    Creates and returns a summarization function.

    Returns:
        function: A function that takes text, max_length, and min_length as input and returns a summary.
    """
    summarizer = pipeline("summarization")

    def summarize(text, max_length=150, min_length=10):
        """
        Summarizes the given text.

        Args:
            text (str): The text to summarize.
            max_length (int): The maximum length of the summary.
            min_length (int): The minimum length of the summary.

        Returns:
            str: The summarized text, or an error message.
        """
        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
            return summary
        except Exception as e:
            return f"Error during summarization: {e}"

    return summarize

c:\Users\Dipender Singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# %pip install textstat

from transformers import pipeline
from textblob import TextBlob
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import textstat

# Download necessary NLTK resources
nltk.download('vader_lexicon', quiet=True)
nlp = spacy.load("en_core_web_sm")

def create_enhanced_analyzer():
    """
    Creates and returns a function that performs summarization, sentiment analysis,
    polarity, subjectivity, and other text analysis.
    """
    summarizer = pipeline("summarization")
    sentiment_analyzer = pipeline("sentiment-analysis")
    sia = SentimentIntensityAnalyzer()

    def analyze_text(text, max_length=100, min_length=10):
        """
        Analyzes the given text, providing summarization, sentiment, polarity, subjectivity, and more.
        """
        results = {}

        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
            results['summary'] = summary
        except Exception as e:
            results['summary'] = f"Error during summarization: {e}"

        try:
            sentiment_result = sentiment_analyzer(text)[0]
            results['sentiment'] = sentiment_result['label']
            results['sentiment_score'] = sentiment_result['score']
        except Exception as e:
            results['sentiment'] = f"Error during sentiment analysis: {e}"
            results['sentiment_score'] = None

        blob = TextBlob(text)
        results['polarity'] = blob.sentiment.polarity
        results['subjectivity'] = blob.sentiment.subjectivity
        results['words'] = len(blob.words)
        results['sentences'] = len(blob.sentences)
        # results['noun_phrases'] = blob.noun_phrases
        # results['word_counts'] = blob.word_counts
        # results['sentiment_detailed'] = [sentence.sentiment for sentence in blob.sentences]

        # Additional Extractions:

        # Named Entity Recognition (NER)
        doc = nlp(text)
        # results['named_entities'] = [(ent.text, ent.label_) for ent in doc.ents]

        # POS Tagging
        # tokens = nltk.word_tokenize(text)
        # results['pos_tags'] = nltk.pos_tag(tokens)

        # Sentiment Intensity (VADER)
        results['vader_sentiment'] = sia.polarity_scores(text)

        # Readability Score
        results['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)

        return results

    return analyze_text

# Example usage:
analyze_function = create_enhanced_analyzer()

text_to_analyze = """
Happiness is a fundamental goal that many people strive to achieve throughout their lives. It is a state of well-being characterized by positive or pleasant emotions ranging from contentment to intense joy, and it is often the ultimate aim of the decisions people make daily.
 However, the concept of happiness is subjective and varies from person to person. Some believe that happiness can be found in material wealth, while others find it in relationships, professional success, or personal fulfillment.

To truly understand happiness, it is essential to recognize that it is not a constant state but rather a journey that requires effort and self-awareness. Aristotle once said, "Happiness is the meaning and the purpose of life, the whole aim and end of human existence," highlighting the importance of happiness as a life goal.
 Achieving happiness involves understanding what it means to be happy and how to create it in one's life. This process is unique to each individual, as everyone has their own specific characteristics and defining traits that influence their path to happiness.

One key aspect of happiness is the realization that it is not solely dependent on external factors such as wealth or status. While money can provide comfort and security, it does not guarantee happiness. The rich are often burdened with stress, anxiety, and insecurity, whereas the poor, despite their lack of material possessions, can find joy in simpler pleasures and stronger social bonds.

Another important factor in achieving happiness is the development of positive relationships. Spending quality time with family and friends, expressing gratitude, and maintaining a positive outlook on life can significantly contribute to one's overall happiness.
 Positive thoughts and a positive mindset attract positive experiences, making it easier to sustain a state of happiness.

Moreover, happiness can be cultivated through habits that promote well-being. Practicing mindfulness, transforming intense emotions into calmness and contentment, and focusing on the present moment can help maintain a state of happiness.
 Additionally, it is crucial to let go of past mistakes and not worry excessively about the future, as dwelling on negative thoughts can hinder one's ability to experience joy.

In conclusion, happiness is a complex and multifaceted concept that requires personal effort and self-reflection. It is not about achieving perfection or external validation but rather about finding contentment and joy in one's life experiences and relationships. By understanding the true meaning of happiness and actively working towards it, individuals can lead a more fulfilling and joyful life.


"""

analysis_results = analyze_function(text_to_analyze)
print(analysis_results)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
question1 = "What has brought you to therapy?"
text1 = input("Enter your answer here")
print("\n")
summary1 = analyze_function(text1)

print("Summary 1:\n", summary1)

In [ ]:
question1 = "Are there any aspects of your daily life that you struggle with right now?"
text2 = input("Enter your answer here")
summary2 = analyze_function(text2, max_length=80, min_length=20) 
print("\nSummary 2:\n", summary2)

In [ ]:
question2 = "Where do you go for support when you need it? Do you have a network of supportive relationships with others?"
text3 = input("Enter your answer here")
summary3 = analyze_function(text3, max_length=80, min_length=20)
print("\nSummary 3:\n", summary3)

In [ ]:
question3 ="How are your relationships with your family? How often are you in touch with them?"
text4 = input("Enter your answer here")
summary4 = analyze_function(text4)
print("\nSummary 4:\n", summary4)